## This notebook merges the remake data from IMDB and TMDB together, then changes a few of the columns to make them easier to use.

In [1]:
import urllib.request
import json
import pandas as pd
from datetime import datetime
import numpy as np
from loguru import logger
from IPython.display import clear_output
import requests
import re
from scipy import stats

tmdb_key = "ad63716b3506edd1aaa3aef6c8ebd46b"

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
sequels_imdb_df = pd.read_csv("SequelOutput.csv")
sequels_imdb_df.head()

,imdb_id,Movie Budget,Movie Box Office US/CA,Movie Box Office Worldwide,Movie Length,Movie Rating,Movie Director Names,Movie Director Links,Movie Cast Names,Movie Cast Links,Movie Awards
0,tt0795421,"$52,000,000 (estimated)","$144,169,664","$694,476,782",1h 48m,6.5,Phyllida Lloyd,['https://www.imdb.com/name/nm1630273/'],"Meryl Streep, Pierce Brosnan, Amanda Seyfried","['https://www.imdb.com/name/nm0000658/', 'http...",Judy Craymer\n\nGary Goetzman\n\nPhyllida Lloy...
1,tt6911608,"$75,000,000 (estimated)","$120,634,935","$402,264,843",1h 54m,6.6,Ol Parker,['https://www.imdb.com/name/nm0662530/'],"Lily James, Amanda Seyfried, Meryl Streep","['https://www.imdb.com/name/nm4141252/', 'http...",Excellence in Contemporary Film\nMichele Clapton
2,tt0078748,"$11,000,000 (estimated)","$81,900,459","$106,285,522",1h 57m,8.5,Ridley Scott,['https://www.imdb.com/name/nm0000631/'],"Sigourney Weaver, Tom Skerritt, John Hurt","['https://www.imdb.com/name/nm0000244/', 'http...","Best Effects, Visual Effects\nH.R. Giger\n\nCa..."
3,tt0090605,"$18,500,000 (estimated)","$85,160,248","$131,060,248",2h 17m,8.4,James Cameron,['https://www.imdb.com/name/nm0000116/'],"Sigourney Weaver, Michael Biehn, Carrie Henn","['https://www.imdb.com/name/nm0000244/', 'http...","Best Effects, Sound Effects Editing\nDon Sharp..."
4,tt0103644,"$50,000,000 (estimated)","$55,473,545","$159,814,498",1h 54m,6.4,David Fincher,['https://www.imdb.com/name/nm0000399/'],"Sigourney Weaver, Charles S. Dutton, Charles D...","['https://www.imdb.com/name/nm0000244/', 'http...","Best Effects, Visual Effects\nRichard Edlund\n..."


In [3]:
sequels_tmdb_df = pd.read_csv("sequels_tmdb_data.csv", index_col=[0])
sequels_tmdb_df.head()

,title,imdb_id,tmdb_id,release_date,is_sequel,runtime,vote_average,vote_count,popularity,budget,revenue,genres,original_language,production_companies,production_countries,collection_id
0,Mamma Mia!,tt0795421,11631,2008-07-03,0,108,6.966,5868,29.600,52000000,609841637,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",en,"[{'id': 34441, 'logo_path': None, 'name': 'Lit...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",458558.0
1,Mamma Mia! Here We Go Again,tt6911608,458423,2018-07-09,1,113,7.136,3039,19.692,75000000,395044706,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",en,"[{'id': 34441, 'logo_path': None, 'name': 'Lit...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",458558.0
2,Alien,tt0078748,348,1979-05-25,0,117,8.142,12773,47.437,11000000,104931801,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",en,"[{'id': 19747, 'logo_path': None, 'name': 'Bra...","[{'iso_3166_1': 'US', 'name': 'United States o...",8091.0
3,Aliens,tt0090605,679,1986-07-18,1,137,7.921,8445,40.290,18500000,183316455,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,"[{'id': 19747, 'logo_path': None, 'name': 'Bra...","[{'iso_3166_1': 'US', 'name': 'United States o...",8091.0
4,Alien³,tt0103644,8077,1992-05-22,1,114,6.348,4894,19.691,50000000,159773545,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",en,"[{'id': 19747, 'logo_path': None, 'name': 'Bra...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",8091.0


In [4]:
def clean_awards(df, award_col_name):
    
    def parse_award_list(award):
        # only keeping first line    
        award = award.split("\n")[0].strip()
        # removing things surrounded by brackets
        award = re.sub("\(.*?\)|\[.*?\]","",award)
        return award   

    all_awards = []
    keywords = ["Best", "Most", "Top", "Worst", "Choice", "Original", "Favorite"]

    for index, row in df.iterrows():
        remade_awards = row[award_col_name]

        if type(remade_awards) == float:
            all_awards.append([])
            continue

        awards = remade_awards.split(",")
        x = list(map(parse_award_list,awards))
        x = list(filter(None, x))

        all_awards.append(x)

    new_list_of_all_awards = []    

    for awards in all_awards:
        new_list_of_awards = []
        for award in awards:   

            for keyword in keywords:
                if keyword in award:
                    new_list_of_awards.append(award)
                    continue
        new_list_of_all_awards.append(new_list_of_awards)
    return new_list_of_all_awards    

In [5]:
sequels_imdb_df["Movie Awards"] = clean_awards(sequels_imdb_df, "Movie Awards")

Adding a column that stores the year a movie was released.

In [6]:
def get_year(date_string):
    if type(date_string) == str:
        return date_string[0:4]
    return None

# replacing NaNs and empty strings with "0000-00-00" so it can be convertd into a year
sequels_tmdb_df["release_date"] = (sequels_tmdb_df["release_date"].fillna("0000-00-00")
                                   .replace(r'^\s*$', "0000-00-00", regex=True))

year_col = sequels_tmdb_df["release_date"].apply(lambda x: get_year(x))
sequels_tmdb_df.insert(4, "release_year", year_col)
sequels_tmdb_df["release_year"].astype(int)

# replacing missing values with NaN
sequels_tmdb_df["release_year"] = sequels_tmdb_df["release_year"].replace("0000", np.nan, regex=True)

# changing year column from int to string 
sequels_tmdb_df["release_year"] = pd.to_numeric(sequels_tmdb_df["release_year"],errors="coerce")

sequels_tmdb_df = sequels_tmdb_df[sequels_tmdb_df["release_year"] < 2023]

Simplifying columns by turning dicts into lists.

In [7]:
def clean_list_of_dicts(list_of_dicts, dict_key):
    new_list = []
    for dict in list_of_dicts:
        new_list.append(dict[dict_key])
    return new_list    

In [8]:
sequels_tmdb_df["production_companies"] = sequels_tmdb_df["production_companies"].apply(lambda x: clean_list_of_dicts(eval(x), "name"))
sequels_tmdb_df["production_countries"] = sequels_tmdb_df["production_countries"].apply(lambda x: clean_list_of_dicts(eval(x), "name"))
sequels_tmdb_df["genres"] = sequels_tmdb_df["genres"].apply(lambda x: clean_list_of_dicts(eval(x), "name"))

Merging IMDB and TMDB data using the IMDB ID.

In [9]:
sequels_df = pd.merge(sequels_tmdb_df, sequels_imdb_df, on = ["imdb_id"])

Putting the sequels and their corresponding original movies on the same row.

In [10]:
sequels_df = pd.merge(sequels_df[sequels_df["is_sequel"] == 0], sequels_df[sequels_df["is_sequel"] == 1], how='left' , on = "collection_id", suffixes = ('_original', '_sequel'))
sequels_df.drop(['is_sequel_original', 'is_sequel_sequel'], inplace=True, axis=1)

In [11]:
sequels_df

,title_original,imdb_id_original,tmdb_id_original,release_date_original,release_year_original,runtime_original,vote_average_original,vote_count_original,popularity_original,budget_original,...,Movie Budget_sequel,Movie Box Office US/CA_sequel,Movie Box Office Worldwide_sequel,Movie Length_sequel,Movie Rating_sequel,Movie Director Names_sequel,Movie Director Links_sequel,Movie Cast Names_sequel,Movie Cast Links_sequel,Movie Awards_sequel
0,Mamma Mia!,tt0795421,11631,2008-07-03,2008.0,108,6.966,5868,29.600,52000000,...,"$75,000,000 (estimated)","$120,634,935","$402,264,843",1h 54m,6.6,Ol Parker,['https://www.imdb.com/name/nm0662530/'],"Lily James, Amanda Seyfried, Meryl Streep","['https://www.imdb.com/name/nm4141252/', 'http...",[]
1,Alien,tt0078748,348,1979-05-25,1979.0,117,8.142,12773,47.437,11000000,...,"$18,500,000 (estimated)","$85,160,248","$131,060,248",2h 17m,8.4,James Cameron,['https://www.imdb.com/name/nm0000116/'],"Sigourney Weaver, Michael Biehn, Carrie Henn","['https://www.imdb.com/name/nm0000244/', 'http...","[Best Effects, Best Effects, Best Actress in a..."
2,Alien,tt0078748,348,1979-05-25,1979.0,117,8.142,12773,47.437,11000000,...,"$50,000,000 (estimated)","$55,473,545","$159,814,498",1h 54m,6.4,David Fincher,['https://www.imdb.com/name/nm0000399/'],"Sigourney Weaver, Charles S. Dutton, Charles D...","['https://www.imdb.com/name/nm0000244/', 'http...",[Best Effects]
3,Alien,tt0078748,348,1979-05-25,1979.0,117,8.142,12773,47.437,11000000,...,"$75,000,000 (estimated)","$47,795,658","$161,376,069",1h 49m,6.2,Jean-Pierre Jeunet,['https://www.imdb.com/name/nm0000466/'],"Sigourney Weaver, Winona Ryder, Dominique Pinon","['https://www.imdb.com/name/nm0000244/', 'http...",[Best DVD Collection]
4,Dr. No,tt0055928,646,1962-10-07,1962.0,110,6.999,3151,26.378,1200000,...,"$2,000,000 (estimated)","$24,796,765","$24,809,727",1h 55m,7.4,Terence Young,['https://www.imdb.com/name/nm0950109/'],"Sean Connery, Robert Shaw, Lotte Lenya","['https://www.imdb.com/name/nm0000125/', 'http...",[Best British Cinematography ]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2074,Çalgı Çengi,tt1865569,64468,2011-02-18,2011.0,90,6.986,71,3.099,0,...,NaN,NaN,"$9,201,129",1h 56m,5.6,Selçuk Aydemir,['https://www.imdb.com/name/nm3252853/'],"Ahmet Kural, Murat Cemcir, Bahri Akin","['https://www.imdb.com/name/nm2899002/', 'http...","[Best Movie Actor, Best Movie Actor]"
2075,Vizontele,tt0270053,57892,2001-02-02,2001.0,110,7.200,138,4.493,0,...,"$3,000,000 (estimated)",NaN,"$13,954,476",1h 51m,6.5,Yilmaz Erdogan,['https://www.imdb.com/name/nm0258784/'],"Yilmaz Erdogan, Demet Akbag, Altan Erkekli","['https://www.imdb.com/name/nm0258784/', 'http...","[Best Film, Best Screenplay, Best Cinematograp..."
2076,Helal Olsun Ali Abi,tt0388104,482483,1963-10-31,1963.0,0,4.000,1,1.371,0,...,"TRL 350,000 (estimated)",NaN,NaN,1h 12m,7.2,Hulki Saner,['https://www.imdb.com/name/nm0762517/'],"Sadri Alisik, Erol Amaç, Cemil Sahbaz","['https://www.imdb.com/name/nm0019699/', 'http...",[]
2077,Magic Carpet Ride,tt0470883,30634,2005-12-22,2005.0,106,6.706,136,5.488,0,...,NaN,NaN,"$10,327,763",2h 1m,6.0,Yilmaz Erdogan,['https://www.imdb.com/name/nm0258784/'],"Yilmaz Erdogan, Kivanç Tatlitug, Ezgi Mola","['https://www.imdb.com/name/nm0258784/', 'http...","[Best Movie Actor, Best Movie, Best Comedy Mov..."


Replacing missing budget/revenue information with NaNs.

In [12]:
sequels_df["budget_original"] = sequels_df["budget_original"].replace(0, np.nan)
sequels_df["budget_sequel"] = sequels_df["budget_sequel"].replace(0, np.nan)
sequels_df["revenue_original"] = sequels_df["revenue_original"].replace(0, np.nan)
sequels_df["revenue_sequel"] = sequels_df["revenue_sequel"].replace(0, np.nan)

Adding z-score columns.

In [13]:
sequels_df["z_vote_average_original"] = stats.zscore(sequels_df["vote_average_original"], nan_policy="omit")
sequels_df["z_vote_count_original"] = stats.zscore(sequels_df["vote_count_original"], nan_policy="omit")
sequels_df["z_popularity_original"] = stats.zscore(sequels_df["popularity_original"], nan_policy="omit")
sequels_df["z_budget_original"] = stats.zscore(sequels_df["budget_original"], nan_policy="omit")
sequels_df["z_revenue_original"] = stats.zscore(sequels_df["revenue_original"], nan_policy="omit")

sequels_df["z_vote_average_sequel"] = stats.zscore(sequels_df["vote_average_sequel"], nan_policy="omit")
sequels_df["z_vote_count_sequel"] = stats.zscore(sequels_df["vote_count_sequel"], nan_policy="omit")
sequels_df["z_popularity_sequel"] = stats.zscore(sequels_df["popularity_sequel"], nan_policy="omit")
sequels_df["z_budget_sequel"] = stats.zscore(sequels_df["budget_sequel"], nan_policy="omit")
sequels_df["z_revenue_sequel"] = stats.zscore(sequels_df["revenue_sequel"], nan_policy="omit")

In [14]:
sequels_df.to_csv("sequels_data.csv")